# Tournament selection

GA order of operation:
1. Calculate the fitness of each chromosome in the current generation.
2. **Select the parents from the current generation that will be used to make the next generation.**
3. Perform crossover of the parents to make children that will be the chromosomes in the next generation.
4. Mutate the children made in step 3.

Now we have the fitness of each chromosome in the current population, we want to use a selection operator to select parents from the current generation that will be used to make the next generation. Here, we will use tournament selection with a tournament size of 3. 

First, we will write the fitness operator as a function:

In [1]:
import numpy as np
import random


# Fitness code as a function
def ackley(chromosome):
    lb_x, ub_x = -5, 5
    len_x = (len(chromosome)//2)
    lb_y, ub_y = -5, 5
    len_y = (len(chromosome)//2)

    precision_x = (ub_x-lb_x)/((2**len_x)-1)
    precision_y = (ub_y-lb_y)/((2**len_y)-1)
    
    z = 0
    t = 1 + (len(chromosome)//2)
    x_bit_sum = 0
    for i in range(len(chromosome)//2):
        x_bit = chromosome[-t]*(2**z)
        x_bit_sum += x_bit
        t += 1
        z += 1
        
    z = 0
    t = 1
    y_bit_sum = 0
    for i in range(len(chromosome)//2):
        y_bit = chromosome[-t]*(2**z)
        y_bit_sum += y_bit
        t += 1
        z += 1
        
    x = (x_bit_sum*precision_x)+lb_x
    y = (y_bit_sum*precision_y)+lb_y
    
    fitness = -20.0 * np.exp(-0.2 * np.sqrt(0.5 * (x**2 + y**2))) - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np.pi * y))) + np.e + 20
    
    return np.array([x, y, fitness])

## Generate a new population
To perform selection, we need a population of chromosomes to work with. Here, we generate a population with 20 chromosomes by shuffling the chromosome we initiate the population with and then stacking them in a numpy array where each row of the population is a different chromosome.

In [2]:
tournament_size = 3
population_size = 20

chromosome = np.array([0,0,1,0,1,0,1,1,1,1,1,1,0,1,1,1,0,0,1,0])
population = np.empty((0,len(chromosome)))

for i in range(population_size):
    random.shuffle(chromosome)
    population = np.vstack((population, chromosome))

print()
print(population)


[[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1.]
 [1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1.]
 [1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1.]
 [0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1.]
 [1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0.]
 [1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0.]
 [0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1.]
 [0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0.]
 [1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1.]
 [0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0.]
 [1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1

## Choosing chromosomes for a tournament

A tournament is made to select a new parent. The number of chromosomes in each tournament is tournament_size. First, we randomly choose the indices of tournament_size (3 in this example) chromosomes using np.random.choice. We do not want duplicates of the same chromosome in a single tournament, so we set replace=False so that a chosen index is removed from the options before the next index is selected. However, when a new tournament is made to select the next parent, all index options are available once again. 

In [3]:
indices_list = np.random.choice(len(population), tournament_size, replace=False)

print(f"Population size: {population_size} - chosen tournament indices: {indices_list}")
print()

Population size: 20 - chosen tournament indices: [4 5 6]



In [4]:
pos_parent_1 = population[indices_list[0]]
pos_parent_2 = population[indices_list[1]]
pos_parent_3 = population[indices_list[2]]

print()
print(f"chromosome {indices_list[0]}: {pos_parent_1}")
print(f"chromosome {indices_list[1]}: {pos_parent_2}")
print(f"chromosome {indices_list[2]}: {pos_parent_3}")


chromosome 4: [1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1.]
chromosome 5: [0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1.]
chromosome 6: [1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0.]


### Fitness
The fitness function is then used to calculate the fitness of each of the chromosomes in the tournament. The fitness function returns 3 values, x, y and the fitness, so we choose index [2] to get the fitness of the parent.

In [5]:
fitness_parent_1 = ackley(pos_parent_1)[2]
fitness_parent_2 = ackley(pos_parent_2)[2]
fitness_parent_3 = ackley(pos_parent_3)[2]

print()
print(f"Fitness of chromosome {indices_list[0]}: {fitness_parent_1}")
print(f"Fitness of chromosome {indices_list[1]}: {fitness_parent_2}")
print(f"Fitness of chromosome {indices_list[2]}: {fitness_parent_3}")


Fitness of chromosome 4: 14.104025320695866
Fitness of chromosome 5: 8.183946443490017
Fitness of chromosome 6: 10.854247817287186


## Selecting the best chromosome
In this case, the parameters to the Ackley function that result in the lowest result are the fittest. We therefore select the chromosome with the minimum value.

In [6]:
min_obj_fn = min(fitness_parent_1, fitness_parent_2, fitness_parent_3)

if min_obj_fn == fitness_parent_1:
    selected_parent = pos_parent_1
elif min_obj_fn == fitness_parent_2:
    selected_parent = pos_parent_2
else:
    selected_parent = pos_parent_3
    
print()
print(f"tournament winner: {selected_parent}")
print(f"min fitness is: {min_obj_fn}")


tournament winner: [0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1.]
min fitness is: 8.183946443490017


# Selection of two parents

The code above was written assuming the tournament size is always 3. When we write a selection function, we want to have the tournament size as one of the input parameters. We will therefore update the code to work with any tournament size. For this, we can make use of list comprehensions in Python, e.g. [i for i in range(3)] = [0, 1, 2]. We can also use numpy.argmin which returns the index of the minimum value in the array.

In [8]:
tournament_size = 3
population_size = 20

chromosome = np.array([0,0,1,0,1,0,1,1,1,1,1,1,0,1,1,1,0,0,1,0])
population = np.empty((0,len(chromosome)))

for i in range(population_size):
    random.shuffle(chromosome)
    population = np.vstack((population, chromosome))

print("Population")
print(population)
print()

indices_list = np.random.choice(len(population), tournament_size, replace=False)

print(f"Population size: {population_size} - chosen tournament indices: {indices_list}")
print()

chosen_parents = np.array([population[indices_list[i]] for i in range(tournament_size)])

print()
for i in range(tournament_size):
    print(f"chromosome {indices_list[i]}: {chosen_parents[i]}")

parent_fitnesses = np.array([ackley(chosen_parents[i])[2] for i in range(tournament_size)])

print()
for i in range(tournament_size):
    print(f"Fitness of chromosome {indices_list[i]}: {parent_fitnesses[i]}")

min_fitness_idx = np.argmin(parent_fitnesses)
    
print()
print(f"Tournament Winner")
print(f"Chromosome {indices_list[min_fitness_idx]}: {chosen_parents[min_fitness_idx]}")
print(f"Chromosome {indices_list[min_fitness_idx]} fitness: {parent_fitnesses[min_fitness_idx]}")


Population
[[1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1.]
 [1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0.]
 [0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
 [1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0.]
 [0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0.]
 [0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1.]
 [0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 

## Selection function
See /es1_1_genetic_algorithm/selection.ipynb to see the above code written as a function that will be used in /es1_1_genetic_algorithm/GA.ipynb.